<a href="https://colab.research.google.com/github/joshuabare/MachineLearning/blob/main/Bare_Building_an_Intelligent_SQL_Query_Interface_Using_Large_Language_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
from google.colab import files
import re




In [ ]:
uploaded = files.upload()
csv_filename = list(uploaded.keys())[0]
df = pd.read_csv(csv_filename)

raw_name = csv_filename.replace(".csv", "")
table_name = re.sub(r"[^a-zA-Z0-9_]", "_", raw_name)

conn = sqlite3.connect("medical.db")
df.to_sql(table_name, conn, if_exists="replace", index=False)
schema = pd.read_sql_query(f"PRAGMA table_info({table_name})", conn)

preview = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 10;", conn)
display(preview)
print("Database created as medical.db with table:", table_name)

Saving medical_database.csv to medical_database (2).csv


,patient_id,name,age,gender,condition,doctor,specialty,appointment_date,duration_minutes,notes
0,1,Sarah Mitchell,54,Female,Hypertension,Dr. Alan Brooks,Cardiology,2025-02-11,30,Follow-up for blood pressure medication adjust...
1,2,David Johnson,63,Male,Diabetes,Dr. Emily Harris,Endocrinology,2025-01-22,40,Reviewed glucose log and modified insulin dose.
2,3,Emily Carter,29,Female,Asthma,Dr. Karen Lee,Family Medicine,2025-03-14,20,Discussed inhaler use and asthma triggers.
3,4,Michael Brown,47,Male,Anxiety,Dr. James Carter,Psychiatry,2025-02-08,45,Reported improvement with therapy and medication.
4,5,Linda Roberts,72,Female,Heart Failure,Dr. Alan Brooks,Cardiology,2025-01-29,50,Fluid status stable; reinforced low-sodium diet.
5,6,Robert Davis,38,Male,Migraine,Dr. Karen Lee,Family Medicine,2025-02-19,25,Medication changed due to persistent headaches.
6,7,Amanda Wilson,33,Female,Thyroid Disorder,Dr. Emily Harris,Endocrinology,2025-03-03,35,TSH levels reviewed; dose adjusted.
7,8,Daniel Thomas,57,Male,Back Pain,Dr. James Carter,Psychiatry,2025-02-16,30,Referred for physical therapy evaluation.
8,9,Patricia Garcia,45,Female,Depression,Dr. James Carter,Psychiatry,2025-03-10,50,Reviewed medication response and coping strate...
9,10,Matthew Miller,61,Male,Arrhythmia,Dr. Alan Brooks,Cardiology,2025-01-18,40,EKG performed; stable rhythm noted.


Database created as medical.db with table: medical_database__2_


In [ ]:
!pip install langchain langchain-community langchain-google-genai sqlalchemy pandas google-generativeai gradio --quiet

import sqlite3
import pandas as pd
import re
import gradio as gr
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain

# Load Gemini API key from Colab secrets
api_key = userdata.get("GEMINI_API_KEY")

# Initialize Gemini model for SQL generation
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,
    temperature=0.1
)

# Connect LangChain to the SQLite database file
db = SQLDatabase.from_uri("sqlite:///medical.db")

# Table name created earlier
table_name = "medical_records"

# Create a chain that converts natural language into SQL queries
sql_chain = create_sql_query_chain(llm, db)

# Remove unwanted prefixes or formatting from generated SQL
def clean_sql(sql_text):
    sql_text = sql_text.strip()
    sql_text = re.sub(r"^SQLQuery:\s*", "", sql_text, flags=re.IGNORECASE)
    sql_text = sql_text.replace("```sql", "").replace("```", "")
    return sql_text.strip()

# Generate a short explanation of the query results
def explain(sql_query, df):
    sql_lower = sql_query.lower()

    # Explanation for COUNT queries
    if "count" in sql_lower:
        value = df.iloc[0, 0]
        return f"There are {value} records that match this question.\n\n"

    # Explanation for selection queries
    if "select" in sql_lower:
        row_count = len(df)
        return f"{row_count} records match this question.\n\n"

    # Default explanation
    return "Here are the results:\n\n"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
def chat_with_db(question):
  try:
    raw_sql = sql_chain.invoke({"question": question})
  except Exception as e:
    return "", f"Error generating SQL String: {e}"
  cleaned_sql = clean_sql(raw_sql)
  try:
    conn = sqlite3.connect("medical.db")
    df = pd.read_sql_query(cleaned_sql, conn)

    explanation = explain(cleaned_sql, df)
    result_text = df.to_string(index=False)
    final_output = explanation + result_text
    return cleaned_sql, final_output
  except Exception as e:
    return cleaned_sql, f"Error executing SQL String: {e}"

with gr.Blocks() as demo:
  gr.Markdown("Chat with your SQLite database using Gemini")
  question = gr.Textbox(label="Ask a quesiton about the data ", lines=2)
  sql_output = gr.Textbox(label="Generated SQL", lines=4)


  answer_output = gr.Textbox(label="Query_result", lines=15, max_lines=30)
  run = gr.Button("Run Queruy")
  run.click(chat_with_db, inputs=question, outputs=[sql_output, answer_output])
  demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c06a7ea2a6b2addc97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
